In [3]:
%%capture installations
pip install MediaPipe

In [4]:
import glob
import cv2
import mediapipe as mp
import pandas as pd
import csv
import json
import numpy as np

In [5]:
TRAIN_IMAGE_FILES = {}
TEST_IMAGE_FILES = {}
for filename in glob.glob('<filepath>/aligned/*'):
    key = filename.split("_")[1]
    if 'test' in filename:
        TEST_IMAGE_FILES.update({key: filename})
    elif 'train' in filename:
        TRAIN_IMAGE_FILES.update({key: filename})
print(len(TRAIN_IMAGE_FILES))
print(len(TEST_IMAGE_FILES))

12271
3068


In [18]:
#function definition - generate_landmarks()
def generate_landmarks(dict_files, isTest=False):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_face_mesh = mp.solutions.face_mesh
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    missing_landmark_samples = {}
    landmarks_dict = {}
    filePath = '<filepath>/facemesh/images_test/' if isTest else '<filepath>/facemesh/images_train/'
    with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
        
         for idx, file in dict_files.items():
            image = cv2.imread(file)
            # Convert the BGR image to RGB before processing.
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            # Print and draw face mesh landmarks on the image.
            if not results.multi_face_landmarks:
                missing_landmark_samples.update({idx: file})
                continue
            annotated_image = image.copy()
            for face_landmarks in results.multi_face_landmarks:
                row = str(face_landmarks).replace('landmark', "").replace('}', "").replace('{', "").replace('\n', "").strip().replace('x: ', '').replace('y: ', '').replace('z: ', '').split('  ')
                row = [float(x) for x in row] 
                landmarks_dict.update({idx: row})
                mp_drawing.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())
            cv2.imwrite(filePath + 'annotated_image_' + str(idx) + '.png', annotated_image)
    
    return landmarks_dict, missing_landmark_samples

In [20]:
landmarks_dict_train, missing_dict_train = generate_landmarks(TRAIN_IMAGE_FILES)
df1 = pd.DataFrame(landmarks_dict_train)
df1_T = df1.T
df1_T.to_csv('<filepath>/facemesh/landmarks_train.csv')
df2 = pd.DataFrame([missing_dict_train])
df2_T = df2.T
df2_T.to_csv('<filepath>/facemesh/missing_train.csv')

landmarks_dict_test, missing_dict_test = generate_landmarks(TEST_IMAGE_FILES, isTest=True)
df3 = pd.DataFrame(landmarks_dict_test)
df3_T = df3.T
df3_T.to_csv('<filepath>/facemesh/landmarks_test.csv')
df4 = pd.DataFrame([missing_dict_test])
df4_T = df4.T
df4_T.to_csv('<filepath>/facemesh/missing_test.csv')




   
       